In [43]:
import gymnasium as gym

In [44]:
env = gym.make("Taxi-v3", render_mode="ansi").env

رنگ ابی جایگاه مسافره رنگ صورتی مقصده زرد هم که خود تاکسیه

In [45]:
obs, info = env.reset()
print(env.render())  # متن محیط رو چاپ می‌کنه

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [46]:
env.reset()
print(env.render())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




یعنی تعداد حالات کارهایی که میتونه بکنه: 1 چپ 2 راست 3 بالا 4 پایین 5 سوارکردن مسافر 6 پیاده کردن مسافر

In [47]:
env.action_space

Discrete(6)

In [48]:
env.observation_space

Discrete(500)

In [81]:
state = env.unwrapped.encode(4, 0, 1, 2)
state

406

In [82]:
env.unwrapped.s = state
print(env.render())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [83]:
state

406

0 پایین

1 بالا

2 شرق

3 غرب

4 سوار

5 پیاده


In [84]:
env.unwrapped.P[state]

{0: [(1.0, 406, -1, False)],
 1: [(1.0, 306, -1, False)],
 2: [(1.0, 406, -1, False)],
 3: [(1.0, 406, -1, False)],
 4: [(1.0, 406, -10, False)],
 5: [(1.0, 406, -10, False)]}

ستون اول احتماله که اینجا مسئله احتمالی نیست همش یکه

ستون دوم استیت بعد از انجام یکی از اون شیش تصمیم مشخصه

ستون سوم ریواردهاست

ستون چهارم حل شدن یا نشدن مسئله بعد از هر کدوم از 6 تا تصمیم

In [85]:
env.unwrapped.s = 306
print(env.render())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [105]:
epochs = 0
penalties, rewards = 0, 0
frames = []
done = False
trunc = False

In [91]:
action = env.action_space.sample()
action

np.int64(1)

In [98]:
env.step(action)

(106,
 -1,
 False,
 False,
 {'prob': 1.0, 'action_mask': array([1, 1, 1, 0, 0, 0], dtype=int8)})

In [99]:
state, reward, done, trunc, _ = env.step(action)

In [100]:
if reward == -10:
    penalties += 1

In [106]:
frames.append({
    'frame': env.render(),
    'state': state,
    'action': action,
    'reward': reward
})

In [109]:
while not done:
    action = env.action_space.sample()
    state, reward, done, trunc, _ = env.step(action)
    if reward == -10:
        penalties += 1
    frames.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward
    })
    epochs += 1
    print(epochs)
if done:
    print("Episode finished")
if trunc:
    print("Episode truncated")

Episode finished


In [110]:
penalties

180

In [111]:
from IPython.display import clear_output
from time import sleep
for i, frame in enumerate(frames):
    clear_output(wait=True)
    print(frame['frame'])
    print('Action: ', frame['action'])
    print('State: ', frame['state'])
    print('Reward: ', frame['reward'])
    sleep(0.2)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Action:  5
State:  410
Reward:  20


In [112]:
env.unwrapped.s = 4

In [113]:
print(env.render())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



In [114]:
env.step(0)

(104,
 -1,
 False,
 False,
 {'prob': 1.0, 'action_mask': array([1, 1, 1, 0, 0, 0], dtype=int8)})

In [115]:
env.unwrapped.s = 104
print(env.render())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)



In [116]:
import numpy as np

In [117]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], shape=(500, 6))

In [119]:
import random
alpha = 0.1
gamma = 0.6
epsilon = 0.1
all_epochs = []
all_penalties = []
for i in range(1, 10000):
    state, info = env.reset()    
    epochs, penalties, reward = 0, 0, 0
    done = False
    trunc = False

    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])
            
        next_state, reward, done, trunc, _ = env.step(action)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        
        if reward == -10:
            penalties += 1
        state = next_state
        epochs += 1
    
    if i % 10 == 0:
#         clear_output(wait=True)
        print('Episode: ', i)
        print('Penalties: ', penalties)
        print('Epochs: ', epochs)

Episode:  10
Penalties:  43
Epochs:  978
Episode:  20
Penalties:  12
Epochs:  358
Episode:  30
Penalties:  17
Epochs:  425
Episode:  40
Penalties:  10
Epochs:  126
Episode:  50
Penalties:  4
Epochs:  164
Episode:  60
Penalties:  10
Epochs:  404
Episode:  70
Penalties:  28
Epochs:  582
Episode:  80
Penalties:  18
Epochs:  679
Episode:  90
Penalties:  3
Epochs:  107
Episode:  100
Penalties:  9
Epochs:  226
Episode:  110
Penalties:  3
Epochs:  68
Episode:  120
Penalties:  6
Epochs:  355
Episode:  130
Penalties:  8
Epochs:  188
Episode:  140
Penalties:  2
Epochs:  134
Episode:  150
Penalties:  9
Epochs:  257
Episode:  160
Penalties:  10
Epochs:  243
Episode:  170
Penalties:  7
Epochs:  233
Episode:  180
Penalties:  5
Epochs:  157
Episode:  190
Penalties:  19
Epochs:  214
Episode:  200
Penalties:  4
Epochs:  157
Episode:  210
Penalties:  4
Epochs:  193
Episode:  220
Penalties:  5
Epochs:  272
Episode:  230
Penalties:  3
Epochs:  103
Episode:  240
Penalties:  3
Epochs:  64
Episode:  250
Pena

In [120]:
q_table

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-2.39707634, -2.34778371, -2.38014103, -2.35217906, -2.27325184,
        -9.59270459],
       [-1.83634469, -1.4839411 , -1.83044852, -1.47530376, -0.7504    ,
        -9.52589021],
       ...,
       [-1.08831672, -0.06936017, -1.08831672, -1.08187074, -1.96      ,
        -3.52189909],
       [-2.08077445, -2.07810897, -2.08509498, -2.07892676, -5.07803037,
        -3.59768893],
       [-0.196     , -0.196     ,  0.20983865, 10.95719423, -1.        ,
        -2.18258358]], shape=(500, 6))

اینجا دیگه یادگیری تموم شده اومدی تو فاز تست یا عمل

In [129]:
total_epochs = 0
total_penalties = 0
state, info = env.reset()
print(env.render())
sleep(3)
epochs, penalties, rewrd = 0, 0, 0
done = False
while not done:
    action = np.argmax(q_table[state])
    state, reward, done, trunc, _ = env.step(action)
    if reward == -10:
        penalties += 1
    epochs += 1
    clear_output(wait=True)
    print(env.render())
    sleep(0.5)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



In [126]:
total_epochs = 0
total_penalties = 0
state, info = env.reset()
epochs, penalties, rewrd = 0, 0, 0
done = False
for i in range(1, 10000):
    state, info = env.reset()
    done = False
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, trunc, _ = env.step(action)
        if reward == -10:
            penalties += 1


KeyboardInterrupt: 

In [ ]:
penalties / 10000

0.0